# AIOK Model Adapter Finetuner DEMO
Model Adapter is a convenient framework can be used to reduce training and inference time, or data labeling cost by efficiently utilizing public advanced models and those datasets from many domains. It mainly contains three components served for different cases: Finetuner, Distiller, and Domain Adapter. 

This demo mainly introduces the usage of Finetuner. Take image classification as an example, it shows how to integrate finetuner with ResNet50 on CIFAR100 dataset. This is a build-in usage, you can find customized detailed demo at [here](./Model_Adapter_Finetuner_customized_resnet50_CIFAR100.ipynb).

# Content

* [Model Adapter Finetuner Overview](#Model-Adapter-Finetuner-Overview)
* [Getting Started](#Getting-Started)
    * [Environment Setup](#Environment-Setup)
    * [Launch Training on baseline](#Launch-Training-on-baseline)
    * [Launch Training with Finetuner](#Launch-Training-with-Finetuner)
    * [Performance](#Performance)

## Model Adapter Finetunner Overview
Finetuner is based on pretraining and finetuning technology, it can transfer knowledge from pretrained model to target model with same network structure. 

Pretrained models usually are generated by pretraining process, which is training specific model  on specific dataset and has been performed by DE-NAS, PyTorch, TensorFlow, or HuggingFace. Finetunner retrieves the pretrained model with same network structure, and copy pretrained weights from pretrained model to corresponding layer of target model, instead of random initialization for target mode. With finetunner, we can greatly improve training speed, and usually achieves better performance.

<img src="../imgs/finetuner.png" width="50%">
<center>Model Adapter Finetuner Structure</center>

# Getting Started

## Environment Setup

1. prepare code
    ``` bash
    git clone https://github.com/intel/e2eAIOK.git
    cd e2eAIOK
    git submodule update --init –recursive
    ```
2. build docker image
   ```
   cd Dockerfile-ubuntu18.04 && docker build -t e2eaiok-pytorch112 . -f DockerfilePytorch112 && cd .. && yes 
   ```
3. run docker
   ``` bash
   docker run -it --name model_adapter --shm-size=10g --privileged --network host \
   -v ${dataset_path}:/home/vmagent/app/data  \
   -v `pwd`:/home/vmagent/app/e2eaiok \
   -w /home/vmagent/app/e2eaiok e2eaiok-pytorch112 /bin/bash 
   ```
4. Run in conda and set up e2eAIOK
   ```bash
   conda activate pytorch-1.12.0
   python setup.py sdist && pip install dist/e2eAIOK-*.*.*.tar.gz
   ```
5. Start the jupyter notebook and tensorboard service
   ``` bash
   nohup jupyter notebook --notebook-dir=/home/vmagent/app/e2eaiok --ip=${hostname} --port=8899 --allow-root &
   nohup tensorboard --logdir /home/vmagent/app/data/tensorboard --host=${hostname} --port=6006 & 
   ```
   Now you can visit demso in `http://${hostname}:8899/`, and see tensorboad log in ` http://${hostname}:6006`.

## Launch training on baseline
First we train a vanilla ResNet50 on CIFAR100.

### Configuration
Create a configuration for ResNet50 with CIFAR100.
```yaml
# basic experiment setting
experiment:
  project: "baseline"
  tag: "cifar100_res50"
output_dir: "/home/vmagent/app/data/model"

### dataset and model setting
data_set: "cifar100"
data_path:  "/home/vmagent/app/data/dataset/cifar"
num_workers: 4

model_type: "resnet50"

## training setting
train_epochs: 1
optimizer: "SGD"
learning_rate: 0.00753
weight_decay: 0.00115
momentum: 0.9

lr_scheduler: "CosineAnnealingLR"
lr_scheduler_config:
    T_max: 200

early_stop: "EarlyStopping"
early_stop_config:
    tolerance_epoch: 15

```
You can also find this configuration at [here](../../../conf/ma/demo/baseline/cifar100_res50.yaml).

### Launch training
**Training resnet50 on CIFAR100 from scratch:**

In [5]:
! python /home/vmagent/app/e2eaiok/e2eAIOK/ModelAdapter/main.py --cfg /home/vmagent/app/e2eaiok/conf/ma/demo/baseline/cifar100_res50.yaml



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

configurations:
{'train_batch_size': 128, 'start_epoch': 0, 'initial_pretrain': '', 'kd': {'temperature': 4}, 'drop_last': False, 'optimizer': 'SGD', 'data_path': '/home/vmagent/app/data/dataset/cifar', 'loss_weight': {'backbone': 1.0, 'distiller': 0.0, 'adapter': 0.0}, 'dkd': {'alpha': 1.0, 'beta': 8.0, 'temperature': 4.0, 'warmup': 20}, 'enable_ipex': False, 'log_interval_step': 10, 'train_epochs': 1, 'metric_threshold': 100.0, 'profiler': False, 'warmup_scheduler_epoch': 0, 'distiller': {'type': '', 'teacher': {'type': '', 'initial_pretrain': '', 'pretrain': '', 'frozen': True}, 'save_logits': False, 'use_saved_logits': False, 

## Launch Training with Finetuner
Then we train ResNet50 on CIFAR100 with Finetuner to show the performance imrpovement.

### Prepare pretrained model 
Download pretrained ResNet50 model on ImageNet21k from [here](https://miil-public-eu.oss-eu-central-1.aliyuncs.com/model-zoo/ImageNet_21K_P/models/resnet50_miil_21k.pth) and put it at `${dataset_path}/pretrained/resnet50_miil_21k.pth`.

### Configuration

Create a configuration for Finetuner with ResNet50 with CIFAR100

```yaml
# basic experiment setting
experiment:
  project: "finetuner"
  tag: "cifar100_res50_PretrainI21k"
  strategy: "OnlyFinetuneStrategy"
output_dir: "/home/vmagent/app/data/model"

### dataset and model setting
data_set: "cifar100"
data_path:  "/home/vmagent/app/data/dataset/cifar"
num_workers: 4
input_size: 112

model_type: "resnet50"

## finetuner setting
finetuner:
    type: "Basic"
    pretrain: '/home/vmagent/app/data/pretrained/resnet50_miil_21k.pth'
    pretrained_num_classes: 11221
    finetuned_lr: 0.00445
    frozen: False

## training setting
train_epochs: 1
optimizer: "SGD"
learning_rate: 0.00753
weight_decay: 0.00115
momentum: 0.9

lr_scheduler: "CosineAnnealingLR"
lr_scheduler_config:
    T_max: 200

early_stop: "EarlyStopping"
early_stop_config:
    tolerance_epoch: 15
```
You can also find this configuration at [here](../../../conf/ma/demo/finetuner/cifar100_res50PretrainI21k.yaml).

### Launch Training with Finetuner
**Training resnet50 on CIFAR100 with Finetuner:**

In [7]:
! python /home/vmagent/app/e2eaiok/e2eAIOK/ModelAdapter/main.py --cfg /home/vmagent/app/e2eaiok/conf/ma/demo/finetuner/cifar100_res50PretrainI21k.yaml



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

See abnormal behavior in dataloader when enable IPEX in PyTorch 1.12, set enable_ipex to False!
configurations:
{'lr_scheduler': 'CosineAnnealingLR', 'pretrain': '', 'eval_epochs': 1, 'criterion': 'CrossEntropyLoss', 'data_set': 'cifar100', 'early_stop_config': {'tolerance_epoch': 15, 'delta': 0.0001, 'is_max': True}, 'dkd': {'alpha': 1.0, 'beta': 8.0, 'temperature': 4.0, 'warmup': 20}, 'output_dir': '/home/vmagent/app/data/model', 'data_path': '/home/vmagent/app/data/dataset/cifar', 'loss_weight': {'backbone': 1.0, 'distiller': 0.0, 'adapter': 0.0}, 'eval_batch_size': 128, 'lr_scheduler_config': {'decay_stages': [], 'decay_patien

2023-02-06 03:59:54 50/391
2023-02-06 04:00:01 60/391
2023-02-06 04:00:08 70/391
[2023-02-06 04:00:14] rank(0) epoch(0) Validation: accuracy = 80.6200;	loss = 0.6625
Best Epoch: 0, accuracy: 80.62000274658203
Epoch 0 took 998.8511202335358 seconds
Total seconds:998.85387
Totally take 1001.8232228755951 seconds


## Performance

We can see the result after training 1 epoch: finetuning achieves **80.62%** validation accuracy, while training from scratch achieves only **7.77%**.

<img src="../imgs/finetuner_result.png" width="80%">
<center>Model Adapter Finetuner Performance</center>